In [22]:
import sqlite3
import pandas as pd
import numpy as np

#Employment Projection Import - Start
empProj = pd.read_excel("EmploymentProjections.xlsx", header=None, skiprows=3, skip_footer=4)
empProj.columns = ['Employment_Title', 'Matrix_Code', 'Occupation', 'Employment_2016', 'Employment_2026', 'Employment_Change_Number', 'Employment_Change_Percent', 'Self_Employed_Percent', 'Openings_2016_2026', 'Median_Wage', 'Education_Needed', 'Work_Experience', 'Job_Training']
empProj = empProj.drop(empProj[empProj.Occupation.str.contains("Summary")].index)
#print(empProj)
#Employment Projection Import - End

#Compensation Change Import - Start
compChange = pd.read_excel("CompensationChange.xlsx", skiprows=15, usecols=['Year', 'Qtr1', 'Qtr2', 'Qtr3', 'Qtr4'])
compChange['Total'] = (compChange['Qtr1'] + compChange['Qtr2'] + compChange['Qtr3'] + compChange['Qtr4']).astype(float)
avgCompChange = compChange['Total'].mean()
#Compensation Change Import - End

#Variables Import - Start
stdVar = pd.read_excel("Variables.xlsx")
stdVar.loc[len(stdVar)] = ['Compensation Change', (avgCompChange / 100)]
#print(stdVar)
#Variables Import - End

#Undergraduate Cost Import - Start
undCost = pd.read_excel("UndergraduateCost.xlsx", header=None, skiprows=6, skip_footer=3)
undCost = undCost.drop(undCost.columns[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20]], axis=1)
undCost.columns = ['Category', 'All_Institutions_2016']
undCost = undCost.drop(undCost[undCost.Category.str.contains("Tuition")==False].index)
undCost = undCost.dropna(subset=['Category'])
undCost = undCost[:-2]
undCost.iloc[-2, undCost.columns.get_loc('Category')] = "4-year institutions"
undCost.iloc[-1, undCost.columns.get_loc('Category')] = "2-year institutions"
#print(undCost)
#Undergraduate Cost Import - End

#Graduate Cost Import - Start
gradCost = pd.read_excel("GraduateCost.xlsx", header=None, skiprows=5, skip_footer=4)
gradCost = gradCost.drop(gradCost.columns[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis=1)
gradCost.columns = ['Year', 'All_Institutions']
gradCost = gradCost.dropna(subset=['Year'])
gradCost = gradCost[:-27]

#Graduate Education Cost Change - For Doctoral Data - Start
eduGrad_2009 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2008-09")].index[0], 'All_Institutions')
eduGrad_2010 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2009-10")].index[0], 'All_Institutions')
eduGrad_2011 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2010-11")].index[0], 'All_Institutions')
eduGrad_2012 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2011-12")].index[0], 'All_Institutions')
eduGrad_2013 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2012-13")].index[0], 'All_Institutions')
eduGrad_2014 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2013-14")].index[0], 'All_Institutions')
eduGrad_2015 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2014-15")].index[0], 'All_Institutions')
eduGrad_2016 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2015-16")].index[0], 'All_Institutions')

eduChange = (eduGrad_2010 / eduGrad_2009) * (eduGrad_2011 / eduGrad_2010) * (eduGrad_2012 / eduGrad_2011) * (eduGrad_2013 / eduGrad_2012) * (eduGrad_2014 / eduGrad_2013) * (eduGrad_2015 / eduGrad_2014) * (eduGrad_2016 / eduGrad_2015)
#Graduate Education Cost Change - For Doctoral Data - End

gradCost = gradCost.drop(gradCost[gradCost.Year.str.contains("2015-16")==False].index)
gradCost.iloc[-1, gradCost.columns.get_loc('Year')] = "2016"
#print(gradCost)
#Graduate Cost Import - End

#Doctoral Cost Import - Start
docCost = pd.read_excel("DoctoralCost.xlsx", header=None, skiprows=5, skip_footer=6)
docCost = docCost.drop(docCost.columns[[2]], axis=1)
docCost.columns = ['Year','Average','Chiropractic', 'Dentistry','Medicine','Optometry','Osteopathic_Medicine','Pharmacy','Podiatry','Veterinary','Law','Theology']
docCost = docCost.drop(docCost[docCost.Year.str.contains("2008-09")==False].index)
docCost = docCost.dropna(subset=['Year'])
docCost = docCost[:-2]

#Doctoral Education Cost Change - Using Graduate Education Change - Start
docCost = docCost[['Average','Chiropractic', 'Dentistry','Medicine','Optometry','Osteopathic_Medicine','Pharmacy','Podiatry','Veterinary','Law','Theology']].multiply(eduChange, axis="index")
docCost.insert(loc=0, column='Year', value=['2016'])
#Doctoral Education Cost Change - Using Graduate Education Change - End
    
#print(docCost)
#Doctoral Cost Import - End

#Tuition Table - Start
tuiTable = pd.DataFrame(columns=['Degree','Type','Education_Years','Annual_Cost'])
tuiTable.loc[0] = ['Undergraduate', 'All', 4, undCost.get_value(undCost[undCost.Category.str.contains("4-year")].index[0], 'All_Institutions_2016')]
tuiTable.loc[len(tuiTable)] = ['Assoicates', 'All', 2, undCost.get_value(undCost[undCost.Category.str.contains("2-year")].index[0], 'All_Institutions_2016')]
tuiTable.loc[len(tuiTable)] = ['Graduate', 'All', 2, gradCost.get_value(gradCost[gradCost.Year.str.contains("2016")].index[0], 'All_Institutions')]
tuiTable.loc[len(tuiTable)] = ['Doctorate', 'Average', 4, docCost.get_value(docCost[docCost.Year.str.contains("2016")].index[0], 'Average')]
tuiTable.loc[len(tuiTable)] = ['Doctorate', 'Chiropractic', 4, docCost.get_value(docCost[docCost.Year.str.contains("2016")].index[0], 'Chiropractic')]
tuiTable.loc[len(tuiTable)] = ['Doctorate', 'Dentistry', 4, docCost.get_value(docCost[docCost.Year.str.contains("2016")].index[0], 'Dentistry')]
tuiTable.loc[len(tuiTable)] = ['Doctorate', 'Medicine', 4, docCost.get_value(docCost[docCost.Year.str.contains("2016")].index[0], 'Medicine')]
tuiTable.loc[len(tuiTable)] = ['Doctorate', 'Optometry', 4, docCost.get_value(docCost[docCost.Year.str.contains("2016")].index[0], 'Optometry')]
tuiTable.loc[len(tuiTable)] = ['Doctorate', 'Pharmacy', 4, docCost.get_value(docCost[docCost.Year.str.contains("2016")].index[0], 'Pharmacy')]
tuiTable.loc[len(tuiTable)] = ['Doctorate', 'Podiatry', 4, docCost.get_value(docCost[docCost.Year.str.contains("2016")].index[0], 'Podiatry')]
tuiTable.loc[len(tuiTable)] = ['Doctorate', 'Veterinary', 4, docCost.get_value(docCost[docCost.Year.str.contains("2016")].index[0], 'Veterinary')]
tuiTable.loc[len(tuiTable)] = ['Doctorate', 'Law', 4, docCost.get_value(docCost[docCost.Year.str.contains("2016")].index[0], 'Law')]
#print(tuiTable)
#Tuition Table - End

timeAnalysis = empProj[['Employment_Title','Employment_2016', 'Employment_2026', 'Openings_2016_2026', 'Median_Wage', 'Education_Needed', 'Work_Experience', 'Job_Training']].copy()
timeAnalysis
i = stdVar.get_value(stdVar[stdVar.Variable.str.contains("Career Years")].index[0], 'Value')
t = 0
while t < i:
    timeAnalysis['CashFlow_' + str(2017 + t)] = 0
    t+=1



print(timeAnalysis)


conn = sqlite3.connect('employment_analysis.db')
cur = conn.cursor() 

empProj.to_sql('employmentProjection', conn, if_exists='replace')
tuiTable.to_sql('tuitionCost', conn, if_exists='replace')
stdVar.to_sql('calculationVariables', conn, if_exists='replace')

cur.close()
conn.close()



                                       Employment_Title  Employment_2016  \
3                                      Chief executives            308.9   
4                       General and operations managers           2263.1   
5                                           Legislators             55.5   
7                   Advertising and promotions managers             31.3   
9                                    Marketing managers            218.3   
10                                       Sales managers            385.5   
11            Public relations and fundraising managers             73.5   
13                     Administrative services managers            281.7   
14            Computer and information systems managers            367.6   
15                                   Financial managers            580.4   
16                       Industrial production managers            170.6   
17                                  Purchasing managers             73.9   
18    Transp

In [16]:
conn = sqlite3.connect('employment_analysis.db')
cur = conn.cursor()

empProjresults = cur.execute('SELECT * FROM employmentProjection LIMIT 3;')

for row in empProjresults.fetchall():
    print(row)
    
undCostresults = cur.execute('SELECT * FROM undergraduateCost LIMIT 2;')

for row in undCostresults.fetchall():
    print(row)
    
gradCostresults = cur.execute('SELECT * FROM graduateCost LIMIT 1;')

for row in gradCostresults.fetchall():
    print(row)
    
docCostresults = cur.execute('SELECT * FROM doctoralCost LIMIT 1;')

for row in docCostresults.fetchall():
    print(row)

#test = pd.read_sql('SELECT * FROM doctoralCost LIMIT 1;', conn)
#print(test)

testresults = cur.execute('SELECT Employment_Title FROM employmentProjection WHERE Education_Needed="Doctoral or professional degree";')

for row in testresults.fetchall():
    print(row)

cur.close()
conn.close()


(3, 'Chief executives', '11-1011', 'Line item', 308.9, 296.8, -12.1, -3.9, '22.8', 20.0, '181210', "Bachelor's degree", '5 years or more', 'None')
(4, 'General and operations managers', '11-1021', 'Line item', 2263.1, 2468.3, 205.2, 9.1, '0.6', 210.7, '99310', "Bachelor's degree", '5 years or more', 'None')
(5, 'Legislators', '11-1031', 'Line item', 55.5, 59.4, 3.9, 7.1, '—', 4.4, '23470', "Bachelor's degree", 'Less than 5 years', 'None')
(7, '4-year institutions', 17181.804648322104)
(26, '2-year institutions', 4161.47898169657)
(31, '2016', 17868.480850081924)
(24, '2016', 17851.38813514861, 31396.353327973244, 49156.48805110744, 39135.4427697134, 28968.584496728883, 44002.432590847355, 27289.27521891786, 35506.36072265488, 23860.073331035073, 38736.54511165488, 15689.850264717597)
('Biochemists and biophysicists',)
('Medical scientists, except epidemiologists',)
('Astronomers',)
('Physicists',)
('Clinical, counseling, and school psychologists',)
('Lawyers',)
('Judicial law clerks',)